Data Science Capstone Project Assignment - Web Page Scraping Toronto Postcode, Borough and Neighborhood data

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Store the required Postcode web page url 
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

# Use the BeautifulSoup package for processing the web page - set the returned data to be in an lxml format
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

# Scrape the Postcode table on the web page
web_table = soup.find('table',{'class':'wikitable sortable'})

# Load the contents of the Postcode table into a text array, use the strip method to remove unwanted characters from the dataset 
table_rows = []
# Loop through the contents of table based on tr and td tags
for table_row in web_table.find_all('tr'):
    table_data = table_row.find_all('td')
    # Convert each table row element to text, strip out unwanted characters and store in a text array
    table_rows.append([i.text.strip() for i in table_data]) 
    
# Isolate the Postcode table column headers (th - table header) and use the strip method to remove unwanted characters from the dataset
table_header = web_table.find_all('th')
# Convert each header list element to text and store in table_header 
table_header = [c.text for c in table_header]
# Loop through each of the characters in the string and use the strip method to remove unwanted characters such as '\n' new line escape codes
table_header = [i.strip() for i in table_header]
#print(table_header)

# Load the table rows and column headers into a dataframe
pdf = pd.DataFrame(data = table_rows, columns = table_header)
#pdf.head()

# Remove the empty top row from the dataframe and reset the index
pdf = pdf.drop([0])
pdf = pdf.reset_index(drop = True)
#pdf.head()

# Only include Boroughs in the dataframe that are assigned a Postcode - Ignore cells with a Borough that are 'Not assigned'
pdf = pdf[pdf.Borough != 'Not assigned']

# Sort the values in the dataframe
pdf.sort_values_by = ['Postcode', 'Borough', 'Neighborhood']

# For a given Postcode that has more than one assigned Neighbourhood, join the cells to form just one Postcode cell separated by a comma
pdf = pdf.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

# If a cell has a Borough but 'Not assigned' to a Neighborhood, then make the Neighborhood the same as the Borough
pdf.loc[pdf.Neighborhood == 'Not assigned', 'Neighborhood'] = pdf.Borough
pdf

pdf.shape

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:44: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access


(103, 3)